In [15]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DeepMEIRL_FC
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld
import torch

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path = 'demo_label/eval/'
file_names = os.listdir(tracks_path)
#read images
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

#parse images
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironmentFromImage(imgs[i],file_names[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 154.53it/s]


In [26]:
model = DeepMEIRL_FC(n_input=4,layers=(16,32,64,32,16))
model.to('cuda')
model.load_state_dict(torch.load('relu_model.pth'))
model.eval()
model.cuda()

DeepMEIRL_FC(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=16, out_features=32, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=32, out_features=64, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ELU(alpha=1.0)
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ELU(alpha=1.0)
    (10): Linear(in_features=16, out_features=1, bias=True)
    (11): ReLU()
  )
)

In [27]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0310_10x10',
                                  rewards_mul=[1,-2,-1,-1],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.95,
                                  model = model)

world_trajGen.ShowLearnedRewards()

In [28]:
world_trajGen.GenerateTrajectoriesWithLearnedReward(traj_count=500,traj_length=20)

100%|██████████| 500/500 [00:00<00:00, 3947.72it/s]


,m,trajs
0,1,"[(9, 3, 8), (8, 3, 7), (7, 3, 6), (6, 3, 5), (..."
1,2,"[(7, 3, 6), (6, 3, 5), (5, 3, 4), (4, 3, 4), (..."
2,3,"[(0, 1, 10), (10, 1, 20), (20, 1, 30), (30, 1,..."
3,4,"[(8, 3, 7), (7, 3, 8), (8, 3, 7), (7, 3, 7), (..."
4,5,"[(2, 1, 12), (12, 1, 13), (13, 3, 12), (12, 1,..."
...,...,...
495,496,"[(8, 3, 9), (9, 3, 8), (8, 3, 9), (9, 3, 8), (..."
496,497,"[(8, 3, 7), (7, 3, 7), (7, 3, 7), (7, 3, 6), (..."
497,498,"[(5, 3, 5), (5, 3, 5), (5, 3, 5), (5, 3, 6), (..."
498,499,"[(9, 3, 8), (8, 3, 9), (9, 3, 19), (19, 3, 9),..."


In [29]:
world_trajGen.ShowTrajs_Learner()